In [12]:
import numpy as np

import keras
import keras.backend as K
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

%config InlineBackend.fugure_format = 'retina'

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
IMAGE_PATCH_SHAPE = (64, 64, 1)

In [14]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Lambda, Flatten, BatchNormalization, Activation
    
# siam from "Learning to Compare Image Patches via Convolutional Neural Networks"
descriptor = Sequential()
descriptor.add(Conv2D(96, kernel_size=7, strides=3, input_shape=IMAGE_PATCH_SHAPE))
descriptor.add(BatchNormalization())
descriptor.add(Activation('relu'))
descriptor.add(MaxPooling2D(pool_size=2, strides=2))
descriptor.add(Dropout(0.2))

descriptor.add(Conv2D(192, kernel_size=5, strides=1))
descriptor.add(BatchNormalization())
descriptor.add(Activation('relu'))
descriptor.add(MaxPooling2D(pool_size=2, strides=2))
descriptor.add(Dropout(0.2))

descriptor.add(Conv2D(256, kernel_size=3, strides=1))
descriptor.add(BatchNormalization())
descriptor.add(Activation('relu'))
descriptor.add(Flatten())
descriptor.add(Lambda(lambda x : tf.nn.l2_normalize(x, 0))) # TODO check normalization

print(descriptor.input_shape, descriptor.output_shape)

InternalError: Failed to create session.

In [4]:
from keras.models import Model
from keras.layers import Input, Concatenate

input_a = Input(shape=IMAGE_PATCH_SHAPE)
input_b = Input(shape=IMAGE_PATCH_SHAPE)

descriptor_a = descriptor(input_a)
descriptor_b = descriptor(input_b)

decision = Concatenate()([descriptor_a, descriptor_b])
decision = Dense(512, activation=relu)(decision)
decision = Dense(1)(decision)
model = Model(inputs=[input_a, input_b], outputs=decision)

print(model.input_shape, model.output_shape)

[(None, 64, 64, 1), (None, 64, 64, 1)] (None, 1)


In [15]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

NameError: name 'model' is not defined

In [ ]:
DATA_ROOT = '../data'
NOTREDAME = f'{DATA_ROOT}/notredame'
LIBERTY = f'{DATA_ROOT}/liberty'

!ls {NOTREDAME}/m50_*.txt

In [6]:
from keras.optimizers import SGD, Adam
from keras.losses import hinge

model.compile(loss=hinge, optimizer=Adam())